# RNNs Sequence to Sequence

Nas RNN $n \times m$ que vimos, a "interpretação" ocorre concomitantemente à "leitura". Muitas vezes, contudo, é melhor esperar primeiro por uma interpretação do todo para, só então, iniciar o processo de decodificação. 

Arquiteturas que usam esta estratégias são as __Seq2seq__. Elas consistem normalmente de duas RNNs, uma codificadora e uma decodificadora, que operam como ilustrado a seguir:

_sequência-entrada_ -> **[codificador]** -> _representação_ -> **[decodificador]** -> _sequência-saída_

Assim, a ideia geral é usar a representação interna de uma rede codificadora para capturar o significado e contexto da entrada. Esta informação é então fornecida para a decodificadora que pode, a partir de um símbolo de partida e da representação da codificadora, ir prevendo a próxima saída decodificada até o fim da sequência.  

Vamos estudar esta rede com uma aplicação em um problema muito comum em países de língua inglesa: soletrar uma palavra a partir de sua pronúncia.

### Soletrando a partir de pronúncias (Mofenas --> Grafenas)

No problema que vamos abordar, queremos traduzir a pronuncia de uma palavra, dada como uma lista de fonemas, para a grafia da palavra. Este problema é mais simples que _fala para texto_ ou _tradução_ (no sentido de não precisarmos de quantidades colossais de dados para ver algo acontecer [:)]); contudo, uma dificuldade aqui é a avaliação na escrita de palavras nunca vistas antes. Isto é díficil porque (1) há muitas pronúncias com várias transcrições razoáveis além de (2) palavras homônicas com transcrições distintas (_read_, no passado e presente, por exemplo). 

### Manipulando os dados...

In [1]:
from __future__ import print_function
import pandas as pd
import numpy as np

from keras.models import Model
from keras.layers import Input, LSTM, Dense

Using TensorFlow backend.


In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

Inicialmente temos que ler o dicionário de fonemas da CMU, _The CMU pronouncing dictionary_.

In [3]:
pdic = pd.read_csv('data/cmudict-compact.csv', comment=';', 
                   header = -1, names = ['word', 'pronunciation'],
                   keep_default_na = False)

In [4]:
len(pdic)

133779

In [6]:
pdic[40150:40160]

,word,pronunciation
40150,FACEY,F EY1 S IY0
40151,FACHET,F AE1 CH AH0 T
40152,FACIAL,F EY1 SH AH0 L
40153,FACIALS,F EY1 SH AH0 L Z
40154,FACIANE,F AA0 S IY0 AA1 N EY0
40155,FACIE,F EY1 S IY0
40156,FACILE,F AE1 S AH0 L
40157,FACILITATE,F AH0 S IH1 L AH0 T EY2 T
40158,FACILITATED,F AH0 S IH1 L AH0 T EY2 T IH0 D
40159,FACILITATES,F AH0 S IH1 L AH0 T EY2 T S


Para manter o problema em um tamanho razoaável, vamos usar apenas uma fração do dicionário de fonemas. 

In [7]:
num_samples = 50000  # Number of samples to train on.
pdic = pdic.sample(n = num_samples)

In [8]:
len(pdic)

50000

In [9]:
pdic.head(5)

,word,pronunciation
62174,JUBILEE(1),JH UW2 B AH0 L IY1
80316,MORGUN,M AO1 R G AH0 N
38198,EPISTEMIC,EH2 P IH0 S T EH1 M IH0 K
62628,KAJUAHAR,K AH0 JH UW1 AH0 HH AA0 R
64535,KIRSCHNER,K ER1 SH N ER0


Em nosso problema, a entrada serão as sequências de fonemas e a saída, as palavras. O script abaixo extrai todas as entradas (listas de fonemas) e alvos (palavras), bem como os conjuntos de símbolos observados nas entradas e alvos (note que os alvos são sempre precedidos de um símbolo que indica início de sequência ['\t'] e terminados em um que indica fim de sequência ['\n']). Ele Ttambém filtra as palavras muito curtas, muito longas ou com símbolos especiais:

In [10]:
def filter_input(inp):    
    return ((len(inp) < 5 or      # filter long words 
             len(inp) > 15) or
            # filter words with not alphabetical chars
            any((not s.isalpha() for s in inp)))

def vectorize(pdic):
    # Vectorize the data.
    input_texts = []
    target_texts = []
    input_symbols = set()
    target_symbols = set()
    for idx, cols in pdic.iterrows():
        target = cols['word']
        if filter_input(target):
            continue
        # We use "tab" as the "start sequence" character
        # for the targets, and "\n" as "end sequence" character.
        target_text = '\t' + target + '\n' # sequence of letters
        target_texts.append(target_text) 
        input_text = cols['pronunciation'].split() # sequence of phonemes
        input_texts.append(input_text)
        for symbol in input_text:
            if symbol not in input_symbols:
                input_symbols.add(symbol)
        for symbol in target_text:
            if symbol not in target_symbols:
                target_symbols.add(symbol)
    input_symbols = sorted(list(input_symbols))
    target_symbols = sorted(list(target_symbols))
    return input_texts, target_texts, input_symbols, target_symbols

In [11]:
input_texts, target_texts, input_symbols, target_symbols = vectorize(pdic)

In [12]:
len(input_texts)

40336

In [15]:
input_texts[0], target_texts[0]

(['M', 'AO1', 'R', 'G', 'AH0', 'N'], '\tMORGUN\n')

In [18]:
len(input_symbols), len(target_symbols)

(69, 28)

In [20]:
num_encoder_tokens = len(input_symbols)
num_decoder_tokens = len(target_symbols)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [21]:
max_encoder_seq_length, max_decoder_seq_length

(16, 17)

Abaixo, vamos criar os mapas que vão fornecer os mapeamentos de cada símbolo para o seu índice correspondente. Com isso, iniciamos os vetores de embeddings que serão usados para representar cada um dos símbolos:

In [22]:
input_token_index = dict(
    [(s, i) for i, s in enumerate(input_symbols)])
target_token_index = dict(
    [(s, i) for i, s in enumerate(target_symbols)])

In [25]:
input_token_index['AE0']

3

In [29]:
target_token_index['A']

2

Nossa arquitetura seq2seq será assim:

<img src="images/rnn_s2s0.png" alt="Exemplo de RNN" style="width: 600px;"/>

Codificação _one-hot-vector_:

In [30]:
def get_hot_vectors(input_texts, target_texts):
    encoder_input_data = np.zeros(
        (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
        dtype='float32')
    decoder_input_data = np.zeros(
        (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
        dtype='float32')
    decoder_target_data = np.zeros(
        (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
        dtype='float32')
    for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
        for t, sym in enumerate(input_text):
            encoder_input_data[i, t, input_token_index[sym]] = 1.
        for t, sym in enumerate(target_text):
            # decoder_target_data is ahead of decoder_target_data by one timestep
            decoder_input_data[i, t, target_token_index[sym]] = 1.
            if t > 0:
                # decoder_target_data will be ahead by one timestep
                # and will not include the start character.
                decoder_target_data[i, t - 1, target_token_index[sym]] = 1.
    return encoder_input_data, decoder_input_data, decoder_target_data

In [31]:
def print_mat_as_map(m):
    if m.shape[1] == num_encoder_tokens:
        symbs = [s.rjust(3, ' ') for s in input_symbols]
        print('   ' + ''.join([s[-3] for s in symbs]))
        print('   ' + ''.join([s[-2] for s in symbs]))
        print('   ' + ''.join([s[-1] for s in symbs]))
    else:
        print('   tnABCDEFGHIJKLMNOPQRSTUVWXYZ')
    for i in range(m.shape[0]):
        print('%2d ' % i, end='')
        for j in range(m.shape[1]):
            print('%s' % '.' if m[i,j]==0 else '*', end='')
        print('\n', end='')

In [32]:
(encoder_input_data, 
 decoder_input_data, 
 decoder_target_data) = get_hot_vectors(input_texts, target_texts)

In [34]:
target_texts[1]

'\tEPISTEMIC\n'

In [38]:
print_mat_as_map(decoder_target_data[1])

   tnABCDEFGHIJKLMNOPQRSTUVWXYZ
 0 ......*.....................
 1 .................*..........
 2 ..........*.................
 3 ....................*.......
 4 .....................*......
 5 ......*.....................
 6 ..............*.............
 7 ..........*.................
 8 ....*.......................
 9 .*..........................
10 ............................
11 ............................
12 ............................
13 ............................
14 ............................
15 ............................
16 ............................


In [39]:
print_mat_as_map(encoder_input_data[1])

   AAAAAAAAAAAAAAAAAA    EEEEEEEEE   IIIIII      OOOOOO      UUUUUU     
   AAAEEEHHHOOOWWWYYY C DHHHRRRYYY  HHHHYYYJ    NWWWYYY   S THHHWWW    Z
   012012012012012012BHDH012012012FGH012012HKLMNG012012PRSHTH012012VWYZH
 0 ........................*............................................
 1 ....................................................*................
 2 ..................................*..................................
 3 ......................................................*..............
 4 ........................................................*............
 5 .......................*.............................................
 6 ...........................................*.........................
 7 ..................................*..................................
 8 .........................................*...........................
 9 .....................................................................
10 ................................................

### Implementando em tensorflow, usando Keras

In [40]:
latent_dim = 256 # 256 LSTM cells
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state = True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

In [41]:
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_state = True, 
                    return_sequences = True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                    initial_state = encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation = 'softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [42]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

<img src="images/rnn_s2s0.png" alt="Exemplo de RNN" style="width: 600px;"/>

In [43]:
batch_size = 64
epochs = 20 # coloquem 6

model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
         batch_size = batch_size, epochs = epochs, validation_split=0.2)
model.save('/tmp/seq2seq.h5')

Train on 32268 samples, validate on 8068 samples
Epoch 1/20
32268/32268 [==============================] - 34s - loss: 1.1981 - val_loss: 1.0355
Epoch 2/20
32268/32268 [==============================] - 30s - loss: 0.9039 - val_loss: 0.8830
Epoch 3/20
32268/32268 [==============================] - 31s - loss: 0.7389 - val_loss: 0.7349
Epoch 4/20
32268/32268 [==============================] - 31s - loss: 0.5871 - val_loss: 0.6253
Epoch 5/20
32268/32268 [==============================] - 32s - loss: 0.4796 - val_loss: 0.4914
Epoch 6/20
32268/32268 [==============================] - 31s - loss: 0.4066 - val_loss: 0.5041
Epoch 7/20
32268/32268 [==============================] - 31s - loss: 0.3559 - val_loss: 0.4121s
Epoch 8/20
32268/32268 [==============================] - 31s - loss: 0.3171 - val_loss: 0.3898
Epoch 9/20
32268/32268 [==============================] - 31s - loss: 0.2855 - val_loss: 0.3482
Epoch 10/20
32268/32268 [==============================] - 31s - loss: 0.2601 - val_lo

### Inferência

Para fazer a inferência, iremos usar a seguinte estratégia:

1. Obtenha o estado do codificador para a sequência de entrada.
2. Inicie com uma sequência alvo de tamanho 1 (apenas o símbolo de início de sequência).
3. Dê o estado do codificador e a sequência criada até agora para o decodificador produzir uma distribuição de probabilidade para o próximo símbolo.
4. Amostre o próximo símbolo usando a distribuição (no exemplo a sequir, é apenas usado argmax).
5. Concatene o símbolo amostrado para a sequêcia alvo
6. Repita desde 1 até encontrar o símbolo de fim de sequência ou alcançar o tamanho máximo de representação da saída.

Note que esta estratégia poderia ter sido usada para treinar a rede também.

Vamos testar o nosso modelo de inferência.

In [44]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [45]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [46]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    start = 0 if decoded_sentence[0] != ' ' else 1
    return decoded_sentence[start:]

In [47]:
print('%15s %15s = %s' % ('Guess', 'Correct', 'Phonemes'))
for seq_index in range(50):
    # Take one sequence (part of the training test)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    inputs = ' '.join(input_texts[seq_index])
    correct = pdic[pdic['pronunciation']==inputs]['word'].iloc[0]
    ok = '+' if decoded_sentence[:-1] == correct else ' '
    print('%15s %15s %s %s'%(decoded_sentence[:-1], correct, ok, inputs))  

          Guess         Correct = Phonemes
         MORGAN          MORGUN   M AO1 R G AH0 N
      EPISTEMIC       EPISTEMIC + EH2 P IH0 S T EH1 M IH0 K
      CAJUAHARA        KAJUAHAR   K AH0 JH UW1 AH0 HH AA0 R
      KIRSCHNER       KIRSCHNER + K ER1 SH N ER0
          LAVOE          LAVEAU   L AH0 V OW1
          ASPEN           ASPEN + AE1 S P AH0 N
     PERSECUTER      PERSECUTOR   P ER1 S AH0 K Y UW2 T ER0
       LEANHART        LIENHART   L IY1 N HH AA2 R T
       FETHERLY       FEATHERLY   F EH1 DH ER0 L IY0
          NEELL           NEILL   N IY1 L
        LINHARS        LINHARES   L IH1 N HH ER0 Z
      ROMESBURG       ROMESBURG + R OW1 M Z B ER0 G
       SPECIALS        SPECIALS + S P EH1 SH AH0 L Z
          CLARA           CLARA + K L AE1 R AH0
         RUDENS        RUDENESS   R UW1 D N AH0 S
          EASON           EASON + IY1 Z AH0 N
     BRUTILIZED      BRUTALIZED   B R UW1 T AH0 L AY2 Z D
          PIERS           PYRES   P AY1 ER0 Z
        REBENMA          REBMAN 